In [ ]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

0	Arm Band Adjustment <br>
1	Band Pull-Down Row <br>
2	Bicep Curl <br>
3	Biceps Curl (band) <br>
4	Burpee <br>
5	Butterfly Sit-up <br>
6	Chest Press (rack)<br>
7	Crunch<br>
8	Device on Table<br>
9	Dip<br>
10	Dumbbell Deadlift Row<br>
11	Dumbbell Row (knee on bench) (label spans both arms)<br>
12	Dumbbell Row (knee on bench) (left arm)<br>
13	Dumbbell Row (knee on bench) (right arm)<br>
14	Dumbbell Squat (hands at side)<br>
15	Dynamic Stretch (at your own pace)<br>
16	Elliptical machine<br>
17	Fast Alternating Punches<br>
18	Jump Rope<br>
19	Jumping Jacks<br>
20	Kettlebell Swing<br>
21	Lateral Raise<br>
22	Lawnmower (left arm)<br>
23	Lawnmower (right arm)<br>
24	Lunge (alternating both legs, weight optional)<br>
25	Medicine Ball Slam<br>
26	Non-Exercise<br>
27	Overhead Triceps Extension<br>
28	Plank<br>
29	Power Boat pose<br>
30	Pushup (knee or foot variation)<br>
31	Pushups<br>
32	Repetitive Stretching<br>
33	Rowing machine<br>
34	Running (treadmill)<br>
35	Russian Twist<br>
36	Seated Back Fly<br>
37	Shoulder Press (dumbbell)<br>
38	Side Plank Left side<br>
39	Side Plank Right side<br>
40	Sit-up (hands positioned behind head)<br>
41	Sit-ups<br>
42	Squat<br>
43	Squat (arms in front of body, parallel to ground)<br>
44	Squat (hands behind head)<br>
45	Squat (kettlebell / goblet)<br>
46	Squat Jump<br>
47	Squat Rack Shoulder Press<br>
48	Static Stretch (at your own pace)<br>
49	Static stretch<br>
50	Tap Right Device<br>
51	Triceps Kickback (knee on bench) (label spans both arms)<br>
52	Triceps Kickback (knee on bench) (left arm)<br>
53	Triceps Kickback (knee on bench) (right arm)<br>
54	Triceps extension (lying down)<br>
55	Two-arm Dumbbell Curl (both arms, not alternating)<br>
56	V-up<br>
57	Walk<br>
58	Walking lunge<br>
59	Wall Ball<br>
60	Wall Squat<br>

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/DACON/train_features.csv')

In [ ]:
target = pd.read_csv('/content/drive/MyDrive/DACON/train_labels.csv')

In [ ]:
train = train.drop(columns = ['id', 'time'], axis = 1)

In [ ]:
train = torch.Tensor(train.values)

In [ ]:
target = target.drop(columns=['label_desc', 'id'] , axis= 1)
target = torch.Tensor(target.values)

In [ ]:
train = train.view(-1,600,6)
target = target.view(-1)

In [ ]:
print('훈련 데이터의 크기 : {}'.format(train.shape))
print('레이블의 크기 : {}'.format(target.shape))

훈련 데이터의 크기 : torch.Size([3125, 600, 6])
레이블의 크기 : torch.Size([3125])


In [ ]:
hidden_size = 6 # hyper parameter
learning_rate = 0.01

In [ ]:
print(train[0][:])
print(target[0])

tensor([[  1.2061,  -0.1794,  -0.1484,  -0.5916, -30.5490, -31.6761],
        [  1.2877,  -0.1990,  -0.1824,   0.3031, -39.1391, -24.9272],
        [  1.3046,  -0.1951,  -0.2534,  -3.6173, -44.1226, -25.0196],
        ...,
        [  1.0256,  -0.2278,  -0.3545,  -2.3342,  25.7687,  18.9321],
        [  1.0316,  -0.3879,  -0.2779,  -9.7107,  28.6977,  20.6316],
        [  1.1382,  -0.4268,  -0.4303, -15.8910,  21.6760,  32.1230]])
tensor(37.)


In [ ]:
class Net(torch.nn.Module):
  def __init__(self , input_size , hidden_size , output_size):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(6 , 8 , batch_first= True)
    self.fc = torch.nn.Linear(4800 , 61 , bias = True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = torch.flatten(x, start_dim = 1)
    x = self.fc(x)
    return x

In [ ]:
net = Net(6, 8, 1)

In [ ]:
outputs = net(train)

In [ ]:
outputs[0]

tensor([-0.8594, -0.3801,  0.1955, -0.3835,  0.6552, -0.0690, -0.8129,  0.6432,
         0.1069,  0.7691,  0.0687, -0.6837, -0.6487, -0.1560, -0.1840, -0.3893,
         0.3040,  0.0683,  0.8214,  0.6989, -0.1266,  0.9858,  0.8214, -0.4296,
        -1.1765,  0.0376,  0.3206, -0.0369, -0.3015, -0.0887,  0.6921,  0.5221,
        -0.2337,  0.2806, -0.1941,  0.8151, -0.7526,  0.2734, -0.4997, -0.3615,
         0.3047,  0.2394, -0.8172, -0.2583,  0.5339, -0.2773,  0.0599, -0.0497,
         0.3320,  0.0673,  0.1664,  0.3268, -0.7064, -0.1776, -0.1027,  0.6203,
        -0.3082, -0.2053,  0.6471,  0.0724,  0.8652], grad_fn=<SelectBackward>)

In [ ]:
target_one_hot = torch.zeros((3125,61))
print(target_one_hot)
print(target)
print(target.unsqueeze(1).long())
target_one_hot.scatter_(1, target.unsqueeze(1).long(),1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([37., 26.,  3.,  ..., 15., 26.,  2.])
tensor([[37],
        [26],
        [ 3],
        ...,
        [15],
        [26],
        [ 2]])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [ ]:
target_one_hot.squeeze()[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(train)
  loss = criterion(outputs.float() , target.long())
  # view를 하는 이유는 Batch 차원 제거를 위해
  loss.backward()
  optimizer.step()

  if i % 10 == 0:
    #print('Epoch {:4d} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
    #       i, cost.item(), accuracy * 100,
    #    ))
    #result = outputs.data.numpy().argmax(axis=1)
    result = torch.argmax(outputs , axis = 1)
    train_acc = torch.sum(result == target)
    train_acc = train_acc / 3125
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", target, '\n')
    print('acc : ' ,train_acc * 100)

0 loss:  4.262021541595459 prediction:  tensor([21, 21, 31,  ..., 20, 30, 47]) true Y:  tensor([37., 26.,  3.,  ..., 15., 26.,  2.]) 

acc :  tensor(1.7920)
10 loss:  0.9327360987663269 prediction:  tensor([37, 26,  3,  ..., 15, 14,  2]) true Y:  tensor([37., 26.,  3.,  ..., 15., 26.,  2.]) 

acc :  tensor(80.6080)
20 loss:  0.21269802749156952 prediction:  tensor([37, 26,  3,  ..., 15, 26,  2]) true Y:  tensor([37., 26.,  3.,  ..., 15., 26.,  2.]) 

acc :  tensor(95.2320)
30 loss:  0.046227965503931046 prediction:  tensor([37, 26,  3,  ..., 15, 26,  2]) true Y:  tensor([37., 26.,  3.,  ..., 15., 26.,  2.]) 

acc :  tensor(99.0400)
40 loss:  0.017347324639558792 prediction:  tensor([37, 26,  3,  ..., 15, 26,  2]) true Y:  tensor([37., 26.,  3.,  ..., 15., 26.,  2.]) 

acc :  tensor(99.6800)
50 loss:  0.009296949952840805 prediction:  tensor([37, 26,  3,  ..., 15, 26,  2]) true Y:  tensor([37., 26.,  3.,  ..., 15., 26.,  2.]) 

acc :  tensor(99.8720)
60 loss:  0.0063407826237380505 pred

In [ ]:
train[0].shape

torch.Size([600, 6])

In [ ]:
with torch.no_grad():
  X_test = train[2]
  Y_test = target[2].long()

  output = net(X_test.view(-1,600,6))
  print('prediction : ', torch.argmax(output, axis = 1), 'answer : ',Y_test)

prediction :  tensor([3]) answer :  tensor(3)


In [ ]:
test_X = pd.read_csv('/content/drive/MyDrive/DACON/test_features.csv')

In [ ]:
test_X

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043
3,3125,3,-0.351750,-0.239870,-0.193053,23.686172,88.608721,13.449771
4,3125,4,-0.312934,-0.123762,-0.318621,20.410071,85.327707,13.884912
...,...,...,...,...,...,...,...,...
469195,3906,595,0.104191,-0.784979,0.639513,-10.475346,14.095361,-190.358982
469196,3906,596,0.103297,-0.758954,0.615687,-25.360272,-8.523018,-180.393291
469197,3906,597,0.128294,-0.749389,0.586184,-27.917723,-23.186245,-162.624160
469198,3906,598,0.104130,-0.692731,0.573397,-27.847980,-30.407555,-138.761676


In [ ]:
test_X = test_X.drop(columns = ['id', 'time'], axis = 1)
test_X = torch.Tensor(test_X.values)

In [ ]:
test_X.shape

torch.Size([469200, 6])

In [ ]:
test_X = test_X.view(-1,600,6)

In [ ]:
with torch.no_grad():
  X_test = test_X
  output = net(X_test.view(-1,600,6))

In [ ]:
test_ = torch.argmax(output, axis = 1)

In [ ]:
test_

tensor([26, 26,  8, 26, 26, 26, 52, 44,  8,  8, 26, 28,  8, 26, 22, 26, 50, 15,
         8, 26,  2, 44, 26, 26, 26, 26,  8, 26, 26, 26, 31, 26,  8, 26, 42, 26,
         8, 26, 26, 57,  3, 26, 26, 26, 26, 26,  8, 20, 26, 30, 26, 26,  8, 26,
         8,  9, 26, 26, 51, 26, 26, 26,  8, 24, 26, 26, 26, 28, 40, 26, 26, 26,
        26,  9, 26,  2, 26, 26, 26, 45, 26, 28, 26, 26,  8, 55, 26,  9, 26, 26,
         8, 30, 26, 60, 24, 40, 28, 26, 26, 26, 26, 26, 26, 55, 34,  8, 26, 54,
        26, 26, 60, 14, 26, 26, 15,  2, 26, 26, 26, 26, 26, 26, 26, 26, 26,  8,
        26,  8, 26,  8, 26,  8, 26, 36, 26, 26, 26,  8,  8, 26, 51, 26,  7, 26,
         8, 26, 26, 21, 13, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,  8, 14, 26,
        11, 26,  8,  8, 26, 26, 26, 26, 26, 26, 26,  7, 26, 37, 42, 24, 26, 26,
        26, 26, 26,  2, 55, 26, 26, 26, 26, 26, 26, 54, 26, 26, 26, 26,  8, 26,
        33, 26, 26, 51, 22, 26, 36, 26, 26, 26,  8, 29, 56, 26, 26, 26, 26, 26,
        26, 26, 36, 20,  8, 26, 26,  8, 

In [ ]:
tmp_zero  = torch.zeros(( 782, 61 ))

In [ ]:
tmp_zero.scatter_(1, test_.unsqueeze(1), 1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
torch.argmax(tmp_zero , axis = 1)

tensor([26, 26,  8, 26, 26, 26, 52, 44,  8,  8, 26, 28,  8, 26, 22, 26, 50, 15,
         8, 26,  2, 44, 26, 26, 26, 26,  8, 26, 26, 26, 31, 26,  8, 26, 42, 26,
         8, 26, 26, 57,  3, 26, 26, 26, 26, 26,  8, 20, 26, 30, 26, 26,  8, 26,
         8,  9, 26, 26, 51, 26, 26, 26,  8, 24, 26, 26, 26, 28, 40, 26, 26, 26,
        26,  9, 26,  2, 26, 26, 26, 45, 26, 28, 26, 26,  8, 55, 26,  9, 26, 26,
         8, 30, 26, 60, 24, 40, 28, 26, 26, 26, 26, 26, 26, 55, 34,  8, 26, 54,
        26, 26, 60, 14, 26, 26, 15,  2, 26, 26, 26, 26, 26, 26, 26, 26, 26,  8,
        26,  8, 26,  8, 26,  8, 26, 36, 26, 26, 26,  8,  8, 26, 51, 26,  7, 26,
         8, 26, 26, 21, 13, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,  8, 14, 26,
        11, 26,  8,  8, 26, 26, 26, 26, 26, 26, 26,  7, 26, 37, 42, 24, 26, 26,
        26, 26, 26,  2, 55, 26, 26, 26, 26, 26, 26, 54, 26, 26, 26, 26,  8, 26,
        33, 26, 26, 51, 22, 26, 36, 26, 26, 26,  8, 29, 56, 26, 26, 26, 26, 26,
        26, 26, 36, 20,  8, 26, 26,  8, 

In [ ]:
test_df = pd.DataFrame(tmp_zero.long())

In [ ]:
test_df['id'] = 0
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,id
0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),0
1,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),0
2,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),0
3,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),0
4,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(

In [ ]:
cols = list(test_df.columns.values)

In [ ]:
cols = cols[-1:] + cols[:-1]

In [ ]:
cols

['id',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [ ]:
test_df = test_df[cols]

In [ ]:
test_df

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0)
1,0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0)
2,0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0)
3,0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0)
4,0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,0,tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(1),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tensor(0),tenso

In [ ]:
test_df.to_csv('/content/drive/MyDrive/DACON/sample_submission.csv', sep= ',', na_rep='NaN')